In [ ]:

#generating the sliding window
def sliding_window(data,stepSize,windowSize):
    for x in range(0,stepSize):
        yield (x, data[x:x + windowSize])
    
#prediction
def load_model(window):
    return loaded_model.predict(window)